# SILVER LAYER SCRIPT

### DATA ACCES USING APP

In [0]:
spark.conf.set("fs.azure.account.auth.type.awprojectdatalakestorage.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.awprojectdatalakestorage.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.awprojectdatalakestorage.dfs.core.windows.net", "##############################")
spark.conf.set("fs.azure.account.oauth2.client.secret.awprojectdatalakestorage.dfs.core.windows.net", "#####################################")
spark.conf.set("fs.azure.account.oauth2.client.endpoint.awprojectdatalakestorage.dfs.core.windows.net", "https://login.microsoftonline.com/#######################/oauth2/token")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

### DATA LOADING

LISTING ALL DIRECTORIES AND FILES IN THE BRONZE LAYER 

In [0]:
dbutils.fs.ls("abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/")

[FileInfo(path='abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Calendar/', name='AdventureWorks_Calendar/', size=0, modificationTime=1748616705000),
 FileInfo(path='abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Customers/', name='AdventureWorks_Customers/', size=0, modificationTime=1748616736000),
 FileInfo(path='abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Product_Categories/', name='AdventureWorks_Product_Categories/', size=0, modificationTime=1748616750000),
 FileInfo(path='abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Product_Subcategories/', name='AdventureWorks_Product_Subcategories/', size=0, modificationTime=1748616767000),
 FileInfo(path='abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Products/', name='AdventureWorks_Products/', size=0, modificationTime=1748616783000),
 FileInfo(path='abfss://bronze@awprojectdatalakestorage.dfs.co

### READING DATA

CALENDAR DATA

In [0]:
df_cal = spark.read.format("csv").option("header", "true")\
            .option("inferSchema", "true")\
                .load("abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Calendar")

CUSTOMERS DATA

In [0]:
df_cus = spark.read.format("csv").option("header", "true")\
            .option("inferSchema", "true")\
                .load("abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Customers")

PRODUCT CATEGORIES

In [0]:
df_procat = spark.read.format("csv").option("header", "true")\
            .option("inferSchema", "true")\
                .load("abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Product_Categories")

PRODUCT SUBCATEGORIES

In [0]:
df_prosubcat = spark.read.format("csv").option("header", "true")\
            .option("inferSchema", "true")\
                .load("abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Product_Subcategories")

PRODUCTS

In [0]:
df_pro = spark.read.format("csv").option("header", "true")\
            .option("inferSchema", "true")\
                .load("abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Products")

RETURNS

In [0]:
df_returns = spark.read.format("csv").option("header", "true")\
            .option("inferSchema", "true")\
                .load("abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Returns")

SALES

In [0]:
df_sales = spark.read.format("csv").option("header", "true")\
            .option("inferSchema", "true")\
                .option("recursiveFileLookup", "true")\
                .load('abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Sales*')

TERRITORIES

In [0]:
df_ter = spark.read.format("csv").option("header", "true")\
            .option("inferSchema", "true")\
                .load("abfss://bronze@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Territories")

# TRANSFORMATIONS AND LOADING TO SILVER LAYER

CALENDAR


In [0]:
df_cal = df_cal.withColumn("Month", month(col('Date')))\
        .withColumn("Year", year(col('Date')))

In [0]:
df_cal.write.format("parquet")\
    .mode('append')\
        .option("path", "abfss://silver@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Calendar")\
            .save()

CUSTOMERS

In [0]:
df_cus = df_cus.withColumn("FullName", concat_ws(' ', col('prefix'), col('FirstName'), col('LastName')))

In [0]:
df_cus.write.format("parquet")\
    .mode('append')\
        .option("path", "abfss://silver@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Customers")\
            .save()

PRODUCT CATEGORIES

In [0]:
df_procat.write.format("parquet")\
    .mode('append')\
        .option("path", "abfss://silver@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Product_Categories")\
            .save()

PRODUCT SUBCATEGORIES

In [0]:
df_prosubcat.write.format("parquet")\
    .mode('append')\
        .option("path", "abfss://silver@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Product_Subcategories")\
            .save()

PRODUCTS

In [0]:
df_pro = df_pro.withColumn('ProductSKU', split(col('ProductSKU'),'-')[0])\
                .withColumn('ProductName', split(col('ProductName'), ' ')[0])

In [0]:
df_pro.write.format("parquet")\
    .mode('append')\
        .option("path", "abfss://silver@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Products")\
            .save()

RETURNS

In [0]:
df_returns.write.format("parquet")\
    .mode('append')\
        .option("path", "abfss://silver@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Returns")\
            .save()

TERRITORIES

In [0]:
df_ter.write.format("parquet")\
    .mode('append')\
        .option("path", "abfss://silver@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Territories")\
            .save()

SALES

In [0]:
df_sales = df_sales.withColumn("StockDate", to_timestamp(col('StockDate')))

In [0]:
df_sales = df_sales.withColumn("OrderNumber", regexp_replace(col('OrderNumber'), 'S', 'T'))

In [0]:
df_sales = df_sales.withColumn("multiply", col('OrderLineItem')*col('OrderQuantity'))

In [0]:
df_sales.write.format("parquet")\
    .mode('append')\
        .option("path", "abfss://silver@awprojectdatalakestorage.dfs.core.windows.net/AdventureWorks_Sales")\
            .save()

###SALES ANALYSIS

In [0]:
df_sales.groupBy('OrderDate').agg(count('OrderNumber').alias('total_order'))

DataFrame[OrderDate: date, total_order: bigint]